In [210]:

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt


In [211]:
np.random.seed(100)

In [212]:
# There are going to be 
 
J = 2

# Across time period 

T = 3


# This might lead to bias see the assumption about J tending to infty and sthen T statoc
# This is the same sort of panel data bias!! Look a bit into microetrics


In [213]:
# All the parameter vector 
theta = np.array([1., 0.2, 0.7, 0.7, 0.2, 0.5, 0.1, 0.05])
beta_0 = 1.
beta_l = 0.2
beta_k = 0.7
beta = theta[1, 0.2, 0.7]
# The coeff in the production function stuff 
alpha = 0.7
sigma_eta = 0.2
simga_v = 0.5
simga_omega = 0.1

# This is needed in the capital accumulation equation 
delta = 0.05

# The parameters for the shock in the labor function 
iota = np.random.normal(0, 0.05, (J, T))
# The random shock given to the labor (as in the estimated shock (idk in my mind this should be 0 
# but maybe i could ask Sabien I guess?))
eta_labor = np.random.normal(0, sigma_eta, (J, T))
eta_production = np.random.normal(0, sigma_eta, (J, T))

# The prices for the companies and the wage
p_eq = np.ones((J, T))
wage = np.ones((J, T))/2

# The capital value 
# The productivity shock has to be drawn from the stationart distribution of an ar(1) process and then it has to
# follow and Ar(1) process to begin with


In [214]:
def production(beta, l, k, prod, eta, J):
    var = np.column_stack((l, k))
    V_0 = np.ones(J)
    inputs = np.column_stack((V_0, var))
    y = inputs@beta + prod + eta
    return y

In [215]:
def log_labor_optimal_error(beta, price, wage, omega, eta, k, iota): 
    beta_0 = beta[0]
    beta_l = beta[1]
    beta_k = beta[2]
    l = (1/(1-beta_l))*(np.log((price*beta_l)/wage)+ beta_0 + omega +iota + eta + beta_k*k)
    return l 

In [216]:
def log_labor_optimal(beta, price, wage, omega, eta, k): 
    beta_0 = beta[0]
    beta_l = beta[1]
    beta_k = beta[2]
    l = (1/(1-beta_l))*(np.log((price*beta_l)/wage)+ beta_0 + omega + eta + beta_k*k)
    return l 

In [217]:
def investment(delta, omega, k):
    # There has to be some structure to the investment decision
    gamma = 0.1 
    # The gamma is set for all the time peruids going forward 
    inv = (delta + gamma*omega)*np.exp(k)
    return inv

In [218]:
# getting the labor when there is an error situation
l_error = log_labor_optimal_error(beta, p_eq, wage, omega_0, eta_labor, k, iota)
print(l_error)
# getting the labor at the true optimality 
l = log_labor_optimal(beta, p_eq, wage, omega_0, eta_labor, k)
print(l)
# Getting the investment decision for each period 
inv = investment(delta, omega_0, k)
print(inv)
# The produictoin at the true labor value no error 
y = production(beta, l, k, omega_0, eta_production, J)
print(y)
# Production at the labor choice manager with error because impoerfect emasure of productivity 
y_error = production(beta, l_error, k, omega_0, eta_production, J)
print(y_error)


[2.12022733 0.75846963]
[2.22958767 0.73705211]
[0.2474839  0.12422086]
[3.0600209  2.03667028]
[3.03814883 2.04095378]


In [233]:
# Generating the productivity process for each of the generated firms
# The productivity shocks and their evolution
omega = np.zeros((J, T))
K = np.zeros((J, T))
inv = np.zeros((J, T))
l_optim = np.zeros
l_error = np.zeros


omega[:,0] = np.random.normal(1, 0.05, J)
K[:, 0] = np.random.normal(1, 0.5, J)
inv[:, 0] = investment(delta, omega[:, 0], K[:, 0])



for t in range(1, T):
    omega[:, t] = alpha * omega[:, t-1] + np.random.normal(0, 0.1, size=J)
    inv[:, t] = investment(delta, omega[:, t], K[:, t])
    K[:, t] = (1-delta)*K[:,t-1] + inv[:,t-1]


print(omega)
print(inv)
print(K)

[[0.99325993 0.58556476 0.26698864]
 [0.9229692  0.62220715 0.53044548]]
[[1.12945306 0.10855648 0.07669886]
 [0.19236915 0.11222072 0.10304455]]
[[2.02335698 3.05164219 3.00761656]
 [0.30150033 0.47879446 0.56707545]]


In [ ]:
# Now in each of these period 